In [1]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal.fake_backends import LocalSimulator


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist
from PatchedMeasCal.bv import bv_circuit_cmap



from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

import mthree

import random

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

In [2]:
def distance_measure(x, bv_value):
    if bv_value in x:
        return 1 - (x[bv_value] / sum(x.values()))
    return 1

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [3]:
n_qubits = 5
n_meas_shots = 16000
n_build_shots = 16000
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 
n_shots_qiskit_partial = n_build_shots // (n_qubits)


n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibmq_lima')

circuit = GHZ_prep(backend)

n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

err_cmap = [[1, 4], [2, 4], [3, 4], [0, 3], [0, 4]]

## Mitigation Methods Build

In [4]:
full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
linear_filter = qiskit_linear(backend, n_qubits, n_shots_qiskit_partial)

mit = mthree.M3Mitigation(backend)
mit.cals_from_system(list(range(n_qubits)), n_build_shots // 2)

tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
tpf_err.build(verbose=True)

tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
tpf.build(verbose=True)

ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f1dca1eb7c0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002


Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s
Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s


## Run with repetitions
If everything above is working, let's crank it out a few times

In [5]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[],
    'm3':[]
}

for _ in range(n_reps):

    target_value = random.randint(0, 2 ** (n_qubits - 1) - 1)
    bv_value = bin((target_value << 1) + 1)[2:].zfill(n_qubits)[::-1]
    bv_string = bin(target_value)[2:].zfill(n_qubits - 1)
    circuit = bv_circuit_cmap(bv_string, n_qubits, backend)
    dist_measure = lambda x: distance_measure(x, bv_value)
    
    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        dist_measure(bare_result)
    )
    results['full'].append(
        dist_measure(full_filter.apply(bare_result))
    )
    results['linear'].append(
        dist_measure(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
        dist_measure(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
        dist_measure(tpf_err.apply(bare_result))
    )
    
    results ['m3'].append(
        dist_measure(mit.apply_correction(bare_result, list(range(n_qubits))).nearest_probability_distribution())
    )
    
    results['aim'].append(dist_measure(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(dist_measure(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(dist_measure(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))
    

ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:clos

In [12]:
results

{'bare': [0.31743750000000004,
  0.32818749999999997,
  0.30500000000000005,
  0.40412499999999996,
  0.3115,
  0.39356250000000004,
  0.38631249999999995,
  0.2920625,
  0.14175000000000004,
  0.46175],
 'full': [0.13687106681647943,
  0.12937649417895924,
  0.16343216917904357,
  0.2634792333807967,
  0.12880991198050318,
  0.1649260873580426,
  0.13530691545399331,
  0.1473488896484655,
  9.53570555850547e-13,
  0.28506007758027885],
 'linear': [0.12692269055551852,
  0.13591520729480677,
  0.13750183565809515,
  0.2326984635039413,
  0.11913042672050633,
  0.15924212003189742,
  0.12093881925491556,
  0.1208336705891927,
  2.723599124010434e-12,
  0.2850994592933095],
 'aim': [0.2579166666666667,
  0.254625,
  0.23962499999999998,
  0.25787499999999997,
  0.2507083333333333,
  0.30900000000000005,
  0.2755416666666667,
  0.2330416666666667,
  0.13037500000000002,
  0.2945833333333333],
 'sim': [0.2874375,
  0.30996875,
  0.28493749999999995,
  0.297125,
  0.28596875,
  0.3132812499

In [9]:
results

{'bare': [0.31743750000000004,
  0.32818749999999997,
  0.30500000000000005,
  0.40412499999999996,
  0.3115,
  0.39356250000000004,
  0.38631249999999995,
  0.2920625,
  0.14175000000000004,
  0.46175],
 'full': [0.13687106681647943,
  0.12937649417895924,
  0.16343216917904357,
  0.2634792333807967,
  0.12880991198050318,
  0.1649260873580426,
  0.13530691545399331,
  0.1473488896484655,
  9.53570555850547e-13,
  0.28506007758027885],
 'linear': [0.12692269055551852,
  0.13591520729480677,
  0.13750183565809515,
  0.2326984635039413,
  0.11913042672050633,
  0.15924212003189742,
  0.12093881925491556,
  0.1208336705891927,
  2.723599124010434e-12,
  0.2850994592933095],
 'aim': [0.2579166666666667,
  0.254625,
  0.23962499999999998,
  0.25787499999999997,
  0.2507083333333333,
  0.30900000000000005,
  0.2755416666666667,
  0.2330416666666667,
  0.13037500000000002,
  0.2945833333333333],
 'sim': [0.2874375,
  0.30996875,
  0.28493749999999995,
  0.297125,
  0.28596875,
  0.3132812499

In [8]:
results

{'bare': [0.14193750000000005,
  0.31756249999999997,
  0.40125,
  0.33799999999999997,
  0.2840625,
  0.21981249999999997,
  0.143625,
  0.31737499999999996,
  0.27831249999999996,
  0.2860625],
 'full': [3.014033467252375e-12,
  0.07619404591343171,
  0.1838421154605523,
  0.06063674125714702,
  0.11448038707783725,
  0.07021017692435583,
  6.529221607820546e-13,
  0.18366020511027992,
  0.06763237419372214,
  0.1227452160626169],
 'linear': [7.179812300250887e-13,
  0.11210590296329181,
  0.14811330758227303,
  0.06954682303667703,
  0.13096818065397564,
  0.055010724809207545,
  2.0254908861261356e-12,
  0.14135615269970137,
  0.09659134912921763,
  0.13001244015005586],
 'aim': [0.12520833333333337,
  0.23158333333333336,
  0.34429166666666666,
  0.29012499999999997,
  0.24783333333333335,
  0.17679166666666668,
  0.12037500000000001,
  0.28745833333333337,
  0.20799999999999996,
  0.23254166666666665],
 'sim': [0.19774999999999998,
  0.28918750000000004,
  0.33384375,
  0.29825,


In [13]:
bv_lima_results = {'bare': [0.31743750000000004,
  0.32818749999999997,
  0.30500000000000005,
  0.40412499999999996,
  0.3115,
  0.39356250000000004,
  0.38631249999999995,
  0.2920625,
  0.14175000000000004,
  0.46175],
 'full': [0.13687106681647943,
  0.12937649417895924,
  0.16343216917904357,
  0.2634792333807967,
  0.12880991198050318,
  0.1649260873580426,
  0.13530691545399331,
  0.1473488896484655,
  9.53570555850547e-13,
  0.28506007758027885],
 'linear': [0.12692269055551852,
  0.13591520729480677,
  0.13750183565809515,
  0.2326984635039413,
  0.11913042672050633,
  0.15924212003189742,
  0.12093881925491556,
  0.1208336705891927,
  2.723599124010434e-12,
  0.2850994592933095],
 'aim': [0.2579166666666667,
  0.254625,
  0.23962499999999998,
  0.25787499999999997,
  0.2507083333333333,
  0.30900000000000005,
  0.2755416666666667,
  0.2330416666666667,
  0.13037500000000002,
  0.2945833333333333],
 'sim': [0.2874375,
  0.30996875,
  0.28493749999999995,
  0.297125,
  0.28596875,
  0.31328124999999996,
  0.2856875,
  0.2885,
  0.1803125,
  0.3203125],
 'jigsaw': [0.0893691270252307,
  0.16955419641175828,
  0.1166183503394661,
  0.19641430570852303,
  0.08778878202956353,
  0.18159037213969742,
  0.16633703008951928,
  0.08399440399255609,
  0.032867835500249076,
  0.2078543769905491],
 'cmc': [0.23922050629824798,
  0.2455569832534975,
  0.20586884644043346,
  0.33379013558062953,
  0.23822495952140366,
  0.31277775577826705,
  0.31325440404418003,
  0.20558140192879204,
  0.08358976450695921,
  0.3944844133662265],
 'cmc_err': [0.22666027816129397,
  0.23203684083749088,
  0.18483361186427527,
  0.2962389249871902,
  0.2262811752205799,
  0.2935279163881831,
  0.3021678548848612,
  0.18398404599335694,
  0.08166216014474958,
  0.37335207605386567],
 'm3': [0.14793512266451703,
  0.14533993190247219,
  0.15125173374726775,
  0.25807587407024324,
  0.14032305803589828,
  0.18861308686185385,
  0.14415928735011185,
  0.13491559546667975,
  0.0,
  0.302044972617751]}

In [14]:
res = bv_lima_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')

bare & $0.33\substack{+0.13 \\ -0.19}$
full & $0.16\substack{+0.13 \\ -0.16}$
linear & $0.14\substack{+0.14 \\ -0.14}$
aim & $0.25\substack{+0.06 \\ -0.12}$
sim & $0.29\substack{+0.03 \\ -0.11}$
jigsaw & $0.13\substack{+0.07 \\ -0.10}$
cmc & $0.26\substack{+0.14 \\ -0.17}$
cmc_err & $0.24\substack{+0.13 \\ -0.16}$
m3 & $0.16\substack{+0.14 \\ -0.16}$
